In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Path to your dataset directory
dataset_dir = r'C:\Users\MSI-PC\Downloads\Ripe & Unripe Fruits'

# Define image dimensions expected by MobileNetV2
img_height, img_width = 224, 224

# Define number of classes in your dataset
num_classes = 2  # Ripe and Unripe

# Create a data generator for training and validation images
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # rescale pixel values to [0,1]
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # split data into training and validation sets
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # specify this is for training
)

valid_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # specify this is for validation
)

# create the base pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 2 classes (ripe and unripe)
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size
)

# Save the model
model.save('fruit_ripeness_detection_model.h5')

Found 623 images belonging to 2 classes.
Found 155 images belonging to 2 classes.
Epoch 1/5


C:\Users\MSI-PC\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 17s 565ms/step - accuracy: 0.5247 - loss: 1.8048 - val_accuracy: 0.7734 - val_loss: 0.4927
Epoch 2/5
 1/19 ━━━━━━━━━━━━━━━━━━━━ 4s 251ms/step - accuracy: 0.8125 - loss: 0.4202

C:\Users\MSI-PC\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8125 - loss: 0.4202 - val_accuracy: 0.7778 - val_loss: 0.5689
Epoch 3/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 472ms/step - accuracy: 0.7434 - loss: 0.5167 - val_accuracy: 0.7422 - val_loss: 0.5207
Epoch 4/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7188 - loss: 0.4872 - val_accuracy: 0.7407 - val_loss: 0.5056
Epoch 5/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 13s 570ms/step - accuracy: 0.8167 - loss: 0.4354 - val_accuracy: 0.8125 - val_loss: 0.4646


In [2]:
import tensorflow as tf
import numpy as np
import io
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Load the saved model
model = tf.keras.models.load_model('fruit_ripeness_detection_model.h5')

# Define image dimensions expected by the model
img_height, img_width = 224, 224

# Function to detect features in the fruit image
def detect_features(img):
    # Convert image to numpy array
    img_np = np.array(img)
    
    # Example feature detection (replace with actual logic)
    # For example, you can detect specific color ranges, textures, etc.
    feature_detected = (img_np[:,:,0] > 150) & (img_np[:,:,1] < 100) & (img_np[:,:,2] < 100)
    
    # Example feature score (replace with actual scoring logic)
    feature_score = np.sum(feature_detected) / feature_detected.size
    
    return feature_score

# Function to preprocess the uploaded image
def preprocess_image(img):
    img = img.convert('RGB')  # Ensure image is in RGB format
    img = ImageOps.exif_transpose(img)  # Handle image orientation
    img = img.resize((img_width, img_height))  # Resize image to model's expected sizing
    img = np.array(img)
    img = img / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to handle file upload and classification
def on_upload_button_clicked(change):
    with out:
        plt.close('all')
        out.clear_output()
        
        # Read uploaded image file
        img = upload_btn.value
        
        if img:
            # Display the uploaded image
            img_widget.clear_output(wait=True)
            with img_widget:
                img_obj = Image.open(io.BytesIO(img[-1]['content']))
                plt.imshow(img_obj)
                plt.axis('off')
                plt.show()

            # Preprocess the image for model prediction
            img = Image.open(io.BytesIO(img[-1]['content']))
            img_preprocessed = preprocess_image(img)
            
            # Detect features in the image
            feature_score = detect_features(img)
            
            # Display feature score
            feature_output.value = f"Feature score: {feature_score:.3f}"
            
            # Classify the image using the model
            if feature_score < 0.025:
                result_label.value = 'Cannot detect fruit'
            elif feature_score <= 0.098:
                result_label.value = 'Unripe'
            else:
                prediction = model.predict(img_preprocessed)
                if prediction[0][0] > prediction[0][1]:
                    result_label.value = 'Ripe'
                else:
                    result_label.value = 'Unripe'
        else:
            result_label.value = 'No image uploaded'

# Create a file upload button
upload_btn = widgets.FileUpload(accept='image/*', multiple=False)

# Create a button to trigger image classification
classify_btn = widgets.Button(description='Classify')
classify_btn.on_click(on_upload_button_clicked)

# Output widgets
out = widgets.Output()
img_widget = widgets.Output()
result_label = widgets.Label()
feature_output = widgets.Label()

# Layout
vbox = widgets.VBox([upload_btn, classify_btn, out, img_widget, feature_output, result_label])

# Display the widgets
display(vbox)
